In [9]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Conv2D, MaxPooling2D

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

data_path = '../sim_data/run1/'

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1



In [2]:
logs = pd.read_csv(data_path + 'driving_log.csv')

logs = logs.values

images = []
measurements = []

images_path = data_path + 'IMG/'

for i in tqdm(range(len(logs))):
    line = logs[i]
    center_path = line[0]
    center_path = images_path + center_path.split('/')[-1]
    center_img = cv2.imread(center_path)
    images.append(center_img)
    
    measure = float(line[3])
    measurements.append(measure)
    
X_train = np.array(images)
y_train = np.array(measurements)
    
    

100%|█████████████████████████████████████████████████████████████████████████████| 4443/4443 [00:04<00:00, 976.90it/s]


In [7]:
model = Sequential()

# Normalization
model.add(Lambda(lambda x: x / 255 - 0.5, input_shape=(160, 320, 3)))

model.add(Conv2D(6,(5,5), activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(6,(5,5), activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(8))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')

Train on 3554 samples, validate on 889 samples
Epoch 1/5
3554/3554 [==============================] - 8s 2ms/step - loss: 9.3574 - val_loss: 0.0301
Epoch 2/5
3554/3554 [==============================] - 7s 2ms/step - loss: 0.0199 - val_loss: 0.0271
Epoch 3/5
3554/3554 [==============================] - 7s 2ms/step - loss: 0.0159 - val_loss: 0.0262
Epoch 4/5
3554/3554 [==============================] - 7s 2ms/step - loss: 0.0137 - val_loss: 0.0270
Epoch 5/5
3554/3554 [==============================] - 7s 2ms/step - loss: 0.0118 - val_loss: 0.0277
